In [17]:
import websocket, json, pprint, talib, numpy
from binance.client import Client
from binance.enums import *
import config
import numpy as np
import pandas as pd
import joblib
import torch
from torch import TheModelC

    "t": 123400000, // Kline start time
    "T": 123460000, // Kline close time
    "s": "BNBBTC",  // Symbol
    "i": "1m",      // Interval
    "f": 100,       // First trade ID
    "L": 200,       // Last trade ID
    "o": "0.0010",  // Open price
    "c": "0.0020",  // Close price
    "h": "0.0025",  // High price
    "l": "0.0015",  // Low price
    "v": "1000",    // Base asset volume
    "n": 100,       // Number of trades
    "x": false,     // Is this kline closed?
    "q": "1.0000",  // Quote asset volume
    "V": "500",     // Taker buy base asset volume
    "Q": "0.500",   // Taker buy quote asset volume
    "B": "123456"   // Ignore


In [41]:
SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

RSI_PERIOD = 14
TRADE_SYMBOL = 'ETHUSD'



in_position = False

client = Client(config.KEY, config.SECRET, tld='us')

scaler = joblib.load('standard_scaler_1m.save')
model = torch.load('rnn_model_1m')


In [33]:
opens = []
closes = []
highs = []
lows = []
volumes = []
qavs = []
numTrades = []
tbbavs = []
tbqavs = []
rsis = []
obvs = []

# def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
#     try:
#         print("sending order")
#         order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
#         print(order)
#     except Exception as e:
#         print("an exception occured - {}".format(e))
#         return False

#     return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    #print('received message')
    json_message = json.loads(message)
    #pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = float(candle['c'])
    open_ = float(candle['o'])
    high = float(candle['h'])
    low = float(candle['l'])
    vol = float(candle['v'])
    qav = float(candle['q'])
    numTrade = float(candle['n'])
    tbbav = float(candle['V'])
    tbqav = float(candle['Q'])
    

    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.insert(0, close)
        volumes.insert(0, vol)
        
#         opens.append(float(open_))
#         highs.append(float(high))
#         lows.append(float(low))
#         qavs.append(float(qav))
#         numTrades.append(float(numTrade))
#         tbbavs.append(float(tbbav))
#         tbqavs.append(float(tbqav))
        
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes[:15])
            np_vols = numpy.array(volumes[:15])
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            obv = talib.OBV(np_closes, np_vols)
            print("all rsis calculated so far")
            print(rsi)
            print("all obvs calculated so far")
            print(obv)
            last_rsi = rsi[-1]
            last_obv = obv[-1]
            print("the current rsi is {}".format(last_rsi))
            
            df = pd.DataFrame(data={"opens": [open_], "highs": [high], "lows": [low], "closes": [close], "volumes": [vol], "qavs": [qav], "numTrades": [numTrade], "tbbavs": [tbbav], "tbqavs": [tbqav], "rsis": [last_rsi],"obvs": [last_obv]})
            scaled_df = scaler.transform(scaled_df)
            
            data = scaled_df.to_numpy()
            data = torch.tensor(data)
            data = data.to(torch.float)
            
            print(model.predict(data), df)
            
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
candle closed at 2614.99
closes
[2614.99]
candle closed at 2613.83
closes
[2613.83, 2614.99]
candle closed at 2613.76
closes
[2613.76, 2613.83, 2614.99]
candle closed at 2615.34
closes
[2615.34, 2613.76, 2613.83, 2614.99]
candle closed at 2614.6
closes
[2614.6, 2615.34, 2613.76, 2613.83, 2614.99]
candle closed at 2613.23
closes
[2613.23, 2614.6, 2615.34, 2613.76, 2613.83, 2614.99]
candle closed at 2610.96
closes
[2610.96, 2613.23, 2614.6, 2615.34, 2613.76, 2613.83, 2614.99]
candle closed at 2615.81
closes
[2615.81, 2610.96, 2613.23, 2614.6, 2615.34, 2613.76, 2613.83, 2614.99]
candle closed at 2618.75
closes
[2618.75, 2615.81, 2610.96, 2613.23, 2614.6, 2615.34, 2613.76, 2613.83, 2614.99]
candle closed at 2619.39
closes
[2619.39, 2618.75, 2615.81, 2610.96, 2613.23, 2614.6, 2615.34, 2613.76, 2613.83, 2614.99]
candle closed at 2618.93
closes
[2618.93, 2619.39, 2618.75, 2615.81, 2610.96, 2613.23, 2614.6, 2615.34, 2613.76, 2613.83, 2614.99]
candle closed at 2619.3
closes
[2

False

In [40]:
df = pd.DataFrame(data={"opens": [1], "highs": [1], "lows": [1], "closes": [1], "volumes": [1], "qavs": [1], "numTrades": [1], "tbbavs": [1], "tbqavs": [1], "rsis": [1],"obvs": [1]})
print(df)
data = df.to_numpy()
data = torch.tensor(data)
data = data.to(torch.float)

model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

   opens  highs  lows  closes  volumes  qavs  numTrades  tbbavs  tbqavs  rsis  \
0      1      1     1       1        1     1          1       1       1     1   

   obvs  
0     1  
OrderedDict([('rnn.weight_ih_l0', tensor([[-9.7356e-02,  2.8937e-02, -3.4995e-02, -9.3279e-02,  1.3936e-01,
         -7.2720e-03,  8.3763e-02,  8.7059e-02,  1.3905e-01, -1.2579e-01,
         -2.6080e-01, -1.1842e+00],
        [-1.2067e-01, -1.9658e-02, -1.2624e-01,  6.2109e-02,  6.7483e-02,
          4.2038e-01, -4.7085e-02,  2.7981e-01,  7.4201e-02,  8.1216e-03,
          2.8657e-02,  2.5943e-01],
        [-4.6067e-02, -5.5862e-02, -9.1829e-02,  2.3932e-02,  1.7966e-02,
         -3.3497e-02, -5.8926e-02,  3.6619e-02,  8.5045e-02,  1.2608e-01,
         -3.9480e-03,  1.5232e+00],
        [-2.3389e-02, -8.6536e-02,  9.8113e-02, -2.5239e-02, -7.2794e-02,
         -5.4686e-02, -1.4326e-01,  1.2974e-02,  2.2607e-01, -5.4922e-02,
          1.4010e-02,  1.5038e+00],
        [ 8.1936e-02,  1.0906e-01,  2.4212e-01,

In [6]:
while(True):
    currCandles = client.get_klines(symbol='ETHUSDT', 
                                    interval=Client.KLINE_INTERVAL_15MINUTE)
    print(len(currCandles))

500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500


KeyboardInterrupt: 